# Importering
Vi begynner med å importere verktøyene vi kommer til å bruke, og lager en dataframe med data fra csv-filen.

In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
df = pd.read_csv('kudos-evaluations-types-v3.csv', sep = ',')

# Visualiseringer med stablet søylediagram
I python-cellene under lager vi tre dataframes (df1, df2 og df3) fylt med såkalt "long form data" som egner seg for visualisering med stablet søylediagram. Vi begynner med å korte ned navnene på datainnsamlingskolonnene ved hjelp av en dict og .rename()-operatoren:

In [5]:
column_rename_dict = {'datainnsamling 1': 'data_1',
                     'datainnsamling 2': 'data_2',
                     'datainnsamling 3': 'data_3',
                     'datainnsamling 4': 'data_4',
                     'datainnsamling 5': 'data_5'}
df.rename(columns=column_rename_dict, inplace=True)

I neste celle skal vi lage en dataframe kalt df1. Her ønsker vi å se på hvordan de forskjellige evalueringstypene er fordelt på sektor, så vi tar disse kolonnene med oss fra df til df1. Vi grupperer på sektor og bruker .value_counts(Normalize = True) for å telle evalueringstypene og normalisere disse (for å finne andel) til dataframe og tilbakestiller index for å unngå multi-index. Vi gir kolonnene norske navn og lager en ekstra kolonne med prosentverdier basert på "andel"-kolonnen.

In [6]:
df1 = df[['sector', 'evaluation_type']]
df1 = df1.groupby('sector').value_counts(normalize = True).to_frame().reset_index()
df1 = df1.rename(columns={df1.columns[0]:'sektor', df1.columns[1]:'type', df1.columns[2]:'andel'})
df1['prosent'] = df1['andel'] * 100

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'

Her lager vi en figur ved hjelp av plotly express. px.bar() tar dataframe og kolonner som argumenter. Vi kan bruker fig.update_layout() og fig.update_xaxes() til å modifisere aspekter ved presentasjonen av grafen. Her har vi lagt til en tittel (og flyttet litt på den), samt rotert sektor-navnene for å gjøre dem mer lesbare.

In [ ]:
fig = px.bar(df1, x="sektor", y='prosent', color="type")
fig.update_layout(title_text='Evalueringstyper fordelt på sektor', title_x=0.42)
fig.update_xaxes(tickangle=40)
fig.show()

I df2 ønsker vi å se på de forskjellige datainnsamlingsmetodene, og hvor stor andel de utgjør i forskjellige typer evalueringer. Vi starter med å lage en mindre dataframe df2 med et utvalg kolonner fra df. Deretter smelter vi df2 for å organisere alle datainnsamlingsmetodene (data_1 - data_5) i samme kolonne (denne blir kalt "value"). Den midlertidige dataframen kaller vi "df2_melted". I linje 8 fjerner vi alle rader som mangler verdier i "value"-kolonnen. Til slutt gjentar vi samme prosess som i df1, der vi grupperer og normaliserer til dataframe, angir nye kolonnenavn og lager en "prosent"-kolonne. 

In [ ]:
df2 = df[['sector', 'evaluation_type', 'data_1', 'data_2', 'data_3', 'data_4', 'data_5']]

id_vars = ['sector', 'evaluation_type']
value_vars = ['data_1', 'data_2', 'data_3', 'data_4', 'data_5']
df2_melted = df2.melt(id_vars = id_vars, value_vars = value_vars)
df2_melted = df2_melted.sort_values(['sector', 'evaluation_type'])
 
df2_melted = df2_melted[df2_melted.value.notnull()]

df2 = df2_melted[['evaluation_type', 'value']]
df2 = df2.groupby('evaluation_type').value_counts(normalize = True).to_frame().reset_index()
df2 = df2.rename(columns={df2.columns[0]:'type', df2.columns[1]:'datainnsamling', df2.columns[2]:'andel'})
df2['prosent'] = df2['andel'] * 100

Her lager vi en figur basert på df2 i plotly:

In [ ]:
fig = px.bar(df2, x='type', y='prosent', color='datainnsamling')
fig.update_layout(title_text='Datainnsamlingsmetoder fordelt på evalueringstype', title_x=0.45)
fig.update_xaxes(tickangle=40)
fig.show()

I df3 er vi interessert i å se på datainnsamlingsmetodene, men denne gangen fordelt på sektor. df3 tar utgangspunkt i df2_melted, så mye av jobben er allerede gjort. Vi grupperer og normaliserer til dataframe, gir nye kolonnenavn og lager en "prosent"-kolonne. 

In [ ]:
df3 = df2_melted[['sector', 'value']] 
df3 = df3.groupby('sector').value_counts(normalize = True).to_frame().reset_index()
df3 = df3.rename(columns={df3.columns[0]:'sektor', df3.columns[1]:'datainnsamling', df3.columns[2]:'andel'})
df3['prosent'] = df3['andel'] * 100

Her lager vi en figur basert på df3 i plotly:

In [ ]:
fig = px.bar(df3, x='sektor', y='prosent', color='datainnsamling')
fig.update_layout(title_text='Datainnsamlingsmetoder fordelt på sektor', title_x=0.45)
fig.update_xaxes(tickangle=40)
fig.show()

# Visualiseringer med gruppert søylediagram

Her vil vi bruke grupperte søylediagram for å se på flere variabler på samme tid. Først er vi interessert i å se på lengdene på evalueringer (minste, gjennomsnittlige og største verdier) fordelt på sektor. I cellen under lager vi en dataframe kalt df5, som inneholder data fra kolonnene "sector" og "length". Vi grupperer på sektor og bruker .agg()-operatoren for å gjøre noen enkle beregninger på dataene i "length" (vi ønsker min, mean og max verdier). 

In [ ]:
df5 = df[['sector','length']]
df5 = df5.groupby('sector').agg({'length': ['mean', 'min', 'max']}).reset_index()

Som resultat av aggregeringen har vi multi-index, men vi vil "flate" den ut til single-index for enkelhets skyld. Deretter sorterer vi verdiene på maks-lengde, slik at vi forhåpentligvis kan se en interessant trend i dataene. 

In [ ]:
df5.columns = [' '.join(col).strip() for col in df5.columns.values]
df5 = df5.sort_values(['length max'])

Her lager vi en figur ved hjelp av plotly (graph objects):

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='min', x=df5['sector'], y=df5['length min']),
    go.Bar(name='mean', x=df5['sector'], y=df5['length mean']),
    go.Bar(name='max', x=df5['sector'], y=df5['length max'])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Lengder på evalueringer fordelt på sektor', title_x=0.5)
fig.update_xaxes(tickangle=40)
fig.show()

Her kan vi gjøre det samme, men bytte ut "sector" med "published_at" for å se utvikling i lengder over tid heller enn basert på sektor:

In [ ]:
df6 = df[['published_at','length']]
df6 = df6.groupby('published_at').agg({'length': ['mean', 'min', 'max']}).reset_index()
df6.columns = [' '.join(col).strip() for col in df6.columns.values]
df6 = df6.sort_values(['length max'])

fig = go.Figure(data=[
    go.Bar(name='min', x=df6['published_at'], y=df6['length min']),
    go.Bar(name='mean', x=df6['published_at'], y=df6['length mean']),
    go.Bar(name='max', x=df6['published_at'], y=df6['length max'])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Lengder på evalueringer over tid', title_x=0.5)
fig.update_xaxes(tickangle=15)
fig.show()

# Visualisering med spredningsplott

Her kan vi benytte dataframen vi lagde tidligere (df6) til å visualisere utviklingen i maks-lengde over tid med plotly, denne gangen med scatterplot (spredningsplott). Vi legger til en "trendline" for å mer tydelig vise tendens i utviklingen: 

In [ ]:
fig = px.scatter(df6, x="published_at", y="length max", trendline="ols", labels={
    'length max': 'makslengde (tegn)', 'published_at': 'år'})
fig.update_layout(title_text = 'Makslengde på evalueringer over tid', title_x = 0.5)
fig.show()

# Visualiseringer med arealdiagram
Ettersom datasettet vi jobber med inneholder datoer, kan det være interessant å se hvordan ting har utviklet seg over tid. Vi starter med å lage en mindre dataframe med bare de relevante kolonnene, og grupperer på årstall. Her har vi ikke normalisert, så vi får visualisert mengden publisering i samme graf. 

In [ ]:
df4 = df[['published_at', 'language']]

df4 = df4.groupby('published_at').value_counts().to_frame().reset_index()
df4 = df4.rename(columns={df4.columns[0]:'år',df4.columns[1]:'språk', df4.columns[2]:'antall evalueringer'})

Her lager vi en figur basert på df4 i plotly:

In [ ]:
fig = px.area(df4, x='år', y='antall evalueringer', color='språk')
fig.update_layout(title_text="Språkfordeling i tidsperspektiv", title_x=0.5)
fig.show()

Her starter vi med samme utgangspunkt som da vi lagde df4, men vi lager et filter som begrenser dataene til rader hvor språket er enten norsk bokmål eller engelsk. Deretter grupperer vi og normaliserer til dataframe, samt gir nye navn til kolonnene. Vi lager også en ny "prosent"-kolonne basert på "andel". 

In [ ]:
df4_2 = df[['published_at', 'language']]
språkfilter = df4_2['language'].isin(['nob', 'eng'])
df4_2 = df4_2[språkfilter]
df4_2 = df4_2.groupby('published_at').value_counts(normalize = True).to_frame().reset_index()
df4_2 = df4_2.rename(columns={df4_2.columns[0]:'år',df4_2.columns[1]:'språk', df4_2.columns[2]:'andel'})
df4_2['prosent'] = df4_2['andel'] * 100

Her lager vi en figur basert på df4_2 i plotly. Mengdene med data fører til noen kraftige svingninger, men vi ser fremdeles en økende tendens for engelskspråklige evalueringer over tid. 

In [ ]:
fig = px.area(df4_2, x='år', y='prosent', color='språk')
fig.update_layout(title_text="Språkfordeling i tidsperspektiv", title_x=0.5)
fig.show()